In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from music21 import converter, instrument, note, stream
import numpy as np

# Function to load MIDI files
def load_midi(midi_file):
    notes = []
    midi = converter.parse(midi_file)
    notes_to_parse = None

    # Parse the notes in the MIDI file
    for element in midi.flat.notes:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
    return notes

# User input for MIDI file
midi_file_path = input("Enter the path for the MIDI file (e.g., 'your_music.mid'): ")
notes = load_midi(midi_file_path)

# Prepare sequences for training
sequence_length = 100
network_input = []
network_output = []

# Create input and output sequences
for i in range(len(notes) - sequence_length):
    seq_in = notes[i:i + sequence_length]
    seq_out = notes[i + sequence_length]
    network_input.append(seq_in)
    network_output.append(seq_out)

# Create a set of unique notes
unique_notes = sorted(set(notes))
note_to_int = {note: number for number, note in enumerate(unique_notes)}

# Convert notes to integer
network_input = [[note_to_int[note] for note in seq] for seq in network_input]
network_output = [note_to_int[note] for note in network_output]

# Pad sequences
network_input = pad_sequences(network_input, maxlen=sequence_length)

# One-hot encode output
network_output = tf.keras.utils.to_categorical(network_output, num_classes=len(unique_notes))

# 2. Build the LSTM Model for Music Generation
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(256, input_shape=(network_input.shape[1], 1), return_sequences=True),
    tf.keras.layers.LSTM(256),
    tf.keras.layers.Dense(len(unique_notes), activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

# 3. Train the Model
model.fit(network_input, network_output, epochs=500, batch_size=64)

# 4. Generate Music using the Trained Model
def generate_music(seed, num_notes=50):
    pattern = [note_to_int[note] for note in seed]
    generated_notes = []

    for _ in range(num_notes):
        prediction_input = pad_sequences([pattern], maxlen=sequence_length)
        prediction_input = np.reshape(prediction_input, (1, sequence_length, 1))
        prediction = model.predict(prediction_input, verbose=0)
        index = np.argmax(prediction)
        result = unique_notes[index]
        generated_notes.append(result)
        pattern.append(index)
        pattern = pattern[1:]

    return generated_notes

# 5. Test the Music Generation
seed_input = input("Enter a seed sequence of notes (e.g., ['C4', 'E4', 'G4']): ")
seed_input = eval(seed_input)  # Converts string input to list
print(generate_music(seed_input, 50))
